In [81]:
from bs4 import BeautifulSoup
import re
import logging
import datetime
import pandas as pd
import sqlite3
LOG_FILENAME = 'html_extract.log'
logging.basicConfig(filename=LOG_FILENAME,level=logging.DEBUG)


df = pd.read_csv("CaseNumber.csv")
case_numbers = df["CaseNumber"].to_list()

con = sqlite3.connect("records.db")
cur = con.cursor()

for c in case_numbers:
    try:

        file = open(f"./Records/{c}/{c}.htm",'r')

        con.execute(f"UPDATE Records SET HasRecordDownloaded = 1 WHERE CaseNumber = \"{c}\"")
        con.commit()

        soup = BeautifulSoup( file, "html.parser")
        tds = soup.select('td')

        for td in tds :
            if re.search("summons", td.text, re.IGNORECASE):

                summons = td
                objs = list(summons.parent)
                summons_date = datetime.datetime.strptime(objs[0].text, '%m/%d/%Y').strftime('%Y-%m-%d')
                summons_path = objs[1].find_all("a", href=True)[0]["href"]
                text_path = re.sub("pdf", "txt", summons_path)
        
                summons_file = open(f"./Records/{c}/{text_path}",'r')
                summons_text = summons_file.read()

                summons_text = re.sub(r"\'", "\'\'", summons_text)
                summons_text = re.sub(r"\"", "\"\"", summons_text)

                cur.execute(f"""
                    UPDATE Records SET
                        SummonsPath = \"{summons_path}\",
                        SummonsText = \"{summons_text}\",
                        SummonsDate = \"{summons_date}\"
                    WHERE CaseNumber = \"{c}\"
                    """)
                
                con.commit()

            if re.search("return on writ", td.text, re.IGNORECASE):

                writ = td
                objs = list(writ.parent)
                writ_date = datetime.datetime.strptime(objs[0].text, '%m/%d/%Y').strftime('%Y-%m-%d')
                writ_path = objs[1].find_all("a", href=True)[0]["href"]
                text_path = re.sub("pdf", "txt", writ_path)
    
                writ_file = open(f"./Records/{c}/{text_path}",'r')
                writ_text = writ_file.read()

                writ_text = re.sub(r"\'", "\'\'", writ_text)
                writ_text = re.sub(r"\"", "\"\"", writ_text)

                cur.execute(f"""
                    UPDATE Records SET
                        WritIssued = 1,
                        WritPath = \"{writ_path}\",
                        WritText = \"{writ_text}\",
                        WritDate = \"{writ_date}\"
                    WHERE CaseNumber = \"{c}\"
                    """)
                
                con.commit()

        if len(tds) > 5:
            last_entry = tds[-5]
            objs = list(tds[-5].parent)

            last_entry_date = datetime.datetime.strptime(objs[0].text, '%m/%d/%Y').strftime('%Y-%m-%d')
            last_entry_action = objs[1].text

            cur.execute(f"""
                    UPDATE Records SET
                        LastFilingEntry = \"{last_entry_action}\",
                        LastFilingDate = \"{last_entry_date}\"
                    WHERE CaseNumber = \"{c}\"
                    """)
            
        con.commit()


    except OSError:
        logging.debug(c)
        con.execute(f"UPDATE Records SET HasRecordDownloaded = 0 WHERE CaseNumber = \"{c}\"")
        con.commit()

con.close()

In [59]:
import ollama
import sqlite3

con = sqlite3.connect("records.db")
cur = con.cursor()

case_numbers = cur.execute("SELECT CaseNumber from Records where DefendantsAddress is null").fetchall()

case_numbers = [c[0] for c in case_numbers]

In [67]:
c = case_numbers[2]

In [68]:
def fetch_document(cursor, doc_type, case_number) -> str:
    return cursor.execute(f"SELECT {doc_type}Text from Records where CaseNumber = \'{case_number}\'").fetchone()[0]

In [69]:
complaint = fetch_document(cur, "Complaint", c)

In [70]:
complaint

'E-FILED\nIN COUNTY CLER\n\n\'\'S OFFICE\n\nPIERCE COUNTY, WASHINGTON\n\nJanuary 03 2017\nKEVIN ST\nCOUNTY CLI\nNO: 17-2-0\nTHE SUPERIOR COURT OF THE STATE OF WASHINGTON\nIN AND FOR THE COUNTY OF PIERCE\nOLEG TKACH, No.\nPlaintiff, COMPLAINT FOR UNLAWFUL\nvs. DETAINER\nWESLEY JOHNSON and\n\nAMBER JOHNSON, et. al..\n\nDefendant(s).\n\nSS OS YS SS SS\n\nCOMES NOW the Plaintiff and for cause of action against the Defendant(s) alleges as\nfollows:\n18\nPlaintiff is the owner, pursuant to a bargain and sale deed as a result of a foreclosure\neffective October 26, 2016 (a copy of which is attached hereto as EXHIBIT A). of the\nfollowing unit Pierce County, Washington:\n18324 80 AVENUE EAST\n\nPUYALLUP, WA 98375\nIL.\n\nDefendant(s) occupied the above-described premises pursuant to an agreement with\n\nCOMPLAINT FOR UNLAWFUL DETAINER - 1 THE GRAY LAW FIRM, P.S.\nJudson C. Gray, Attorney\n4142 - 6th Avenue\nTacoma, Washington 98406\nTelephone (253) 759-1141 Facsimile (253) 759-1447\njgravyatto

In [71]:
rent_owed_instruction = "Based on the Complaint above, what is the total owed by the defendant to the landlord? Respond with only the total amount, in the following format: XXX.XX."

defendant_address_instruction = "Based on the Complaint above, what is the defendant's address? Only provide the address."

monthly_rent_instruction = "Based on the Complaint above, what is the defendant's monthly rent? Respond in the amount in the format XXX.XX."

def extract_info(source_text, instructions):

    response = ollama.chat(model='llama3.2', messages=[
    {
        'role': 'user',
        'content': f'{source_text}\n\n{instructions}',
    },
    ])
    
    return response['message']['content']

In [72]:
defendant_address = extract_info(complaint, defendant_address_instruction)

In [73]:
defendant_address

'18324 80th Ave E, Unit # Puyallup WA 98375'

In [ ]:
cur.execute(f"UPDATE Records SET DefendantsAddress = \'{defendant_address}\' ")